In [1]:
import gc
gc.collect()

170

# Importing Library and Instantiating Model

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "lm-studio"
os.environ["OPENAI_MODEL_NAME"] = 'medgemma-4b-it'
os.environ["OPENAI_API_BASE"] = "http://localhost:1234/v1"

In [37]:
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool

# Role Playing, Focus and Co-operation

In [10]:
support_agent = Agent(
    role="Senior Support Representative",
    goal="Be the most friendly and helpful support representative in your team.",
    backstory=(
        "You work at CrewAI (https://crewai.com) and are now working on providing support to {customer}, a super important customer for your company"
        "You need to make sure that you provide the best support! Make sure to provide full complete answers, and make no assumptions."
    ),
    allow_delegation=False,
    verbose=True
)

support_agent

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers



Agent(role=Senior Support Representative, goal=Be the most friendly and helpful support representative in your team., backstory=You work at CrewAI (https://crewai.com) and are now working on providing support to {customer}, a super important customer for your companyYou need to make sure that you provide the best support! Make sure to provide full complete answers, and make no assumptions.)

In [12]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

support_quality_assurance_agent

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers



Agent(role=Support Quality Assurance Specialist, goal=Get recognition for providing the best support quality assurance in your team, backstory=You work at crewAI (https://crewai.com) and are now working with your team on a request from {customer} ensuring that the support representative is providing the best support possible.
You need to make sure that the support representative is providing fullcomplete answers, and make no assumptions.)

# Tools, Guardrails and Memory

In [16]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

In [ ]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)
docs_scrape_tool

ScrapeWebsiteTool(name='Read website content', description="Read website content() - A tool that can be used to read https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/'s content.", args_schema=<class 'crewai_tools.tools.scrape_website_tool.scrape_website_tool.FixedScrapeWebsiteToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x0000019A8C0DFCE0>, website_url='https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/', cookies=None, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 'Accept-Language': 'en-US,en;q=0.9', 'Referer': 'https://www.google.com/', 'Connection': 'keep-alive', 'Upgrade-Insecure-Requests': '1', 'Accept-Encoding': 'gzip, deflate, br'})

In [58]:
from typing import ClassVar
from pydantic import ConfigDict
class MyTool(BaseTool):
    model_config = ConfigDict(ignored_types=(str,))
    name="docs_scrape_tool"
    description="Website Scraping Tool"

    def _run(self, website_url):
        return website_url

docs_scrape_tool_ud = MyTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)
docs_scrape_tool_ud

MyTool(name='docs_scrape_tool', description='Tool Name: docs_scrape_tool\nTool Arguments: {}\nTool Description: Website Scraping Tool', args_schema=<class 'abc.MyToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x0000019AFE651300>, result_as_answer=False)

In [59]:
from crewai.tools import BaseTool
isinstance(docs_scrape_tool_ud, BaseTool)

True

# Tasks

In [60]:
inquiry_resolution = Task(
    description=
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ,
    expected_output=
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ,
	tools=[docs_scrape_tool_ud],
    agent=support_agent,
)

inquiry_resolution

Task(description={customer} just reached out with a super important ask:
{inquiry}

{person} from {customer} is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry., expected_output=A detailed, informative response to the customer's inquiry that addresses all aspects of their question.
The response should include references to everything you used to find the answer, including external data or solutions. Ensure the answer is complete, leaving no questions unanswered, and maintain a helpful and friendly tone throughout.)

In [61]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)
quality_assurance_review

Task(description=Review the response drafted by the Senior Support Representative for {customer}'s inquiry. Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer support.
Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly tone.
Check for references and sources used to  find the information, ensuring the response is well-supported and leaves no questions unanswered., expected_output=A final, detailed, and informative response ready to be sent to the customer.
This response should fully address the customer's inquiry, incorporating all relevant feedback and improvements.
Don't be too formal, we are a chill and cool company but maintain a professional and friendly tone throughout.)

# Memory

In [62]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
)
crew

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



Crew(id=41f439e2-0358-4201-8e18-698bffbbf71c, process=Process.sequential, number_of_agents=2, number_of_tasks=2)

# Crew

In [63]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



ValueError: Required function parameter 'website_url' not found in args_schema

In [ ]:
from IPython.display import Markdown
Markdown(result)